In [1]:
import pandas as pd

products = pd.read_parquet('products.parquet')
return_reason = pd.read_parquet('return_reasons.parquet')
returns = pd.read_parquet('returns.parquet')
reviews = pd.read_parquet('reviews.parquet')
test = pd.read_parquet('test.parquet')
reviews = reviews.dropna(subset = 'review_text')
reviews = reviews[reviews['review_text'] != '']

In [2]:
# take 30 product_id from test and its reviews from reviews dataframe
unique_pds =returns.drop_duplicates('product_id')
mask = ~(test['product_id'].isin(unique_pds['product_id']))
test_kk = test[mask].drop_duplicates('product_id')[:30]
mask2 = reviews['product_id'].isin(test_kk['product_id'])
review_test = reviews[mask2]
test_df = test[test['product_id'].isin(test_kk['product_id'])]
# test_df.nunique()

In [3]:
review_test = review_test.dropna(subset = 'review_text')
review_test = review_test[review_test['review_text'] != '']

In [4]:
# from googletrans import Translator
# df = review_test
# # Create a Translator object
# translator = Translator()
# def query(text):
#     result = translator.translate(text, dest='en')
#     return result.text
# df['translation'] = df['review_text'].apply(query)

In [6]:
# csv_file_path = 'review_test.csv'  # Desired file name
# df.to_csv(csv_file_path, index=True)

In [7]:
review_test = pd.read_csv('review_test.csv')

In [8]:
import nltk 
 
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
 
from nltk.corpus import stopwords 
 
from nltk.tokenize import word_tokenize 
 
from nltk.stem import WordNetLemmatizer 
 
import nltk 
 
nltk.download('all') 
 
 
 
def preprocess_text(text): 
 
    tokens = word_tokenize(text.lower()) 
 
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')] 
 
    lemmatizer = WordNetLemmatizer() 
 
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens] 
 
    processed_text = ' '.join(lemmatized_tokens) 
 
    return processed_text 


analyzer = SentimentIntensityAnalyzer() 
 
def get_sentiment(text): 
 
    scores = analyzer.polarity_scores(text) 
 
    return scores 


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\dustm\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\dustm\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\dustm\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\dustm\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\dustm\AppData\Roaming\nltk_data...
[nltk_data]    | 

In [9]:
def get_sentiment(text): 
 
    scores = analyzer.polarity_scores(text) 
 
    return scores 

In [10]:
review_test['preprop'] = review_test['translation'].apply(preprocess_text) 
review_test['sentiment'] =review_test['preprop'].apply(get_sentiment)
review_test['sentiment'] = review_test['sentiment'].apply(lambda x: x['compound'])

In [11]:
def pos(n):
    if n > 0:
        return n
    else:
        return 0
review_test['positive'] = review_test['sentiment'].apply(pos)

In [12]:
def neg(n):
    if n < 0:
        return n
    else:
        return 0
review_test['negative'] = review_test['sentiment'].apply(neg)

In [13]:
def net(n):
    if n == 0:
        return n
    else:
        return 0
review_test['neutral'] = review_test['sentiment'].apply(net)

In [14]:
dic ={'rating': 'mean', 'sentiment':"mean", 'positive':'mean', 'negative': 'mean', 'neutral': 'mean'}
gr = review_test.groupby(by = 'product_id').agg(dic).reset_index(drop=False)

In [15]:
gr_2 = test_df.groupby(by = 'product_id').agg({'purchase_price': 'mean'})
with_mean_price = gr_2.merge(test_df, on = 'product_id', how = 'right')
with_mean_price = with_mean_price.drop('purchase_price_y', axis = 1)
with_mean_price.nunique()

product_id           30
purchase_price_x     29
id                  153
order_item_id       153
customer_id         124
dtype: int64

In [16]:
test_ = with_mean_price.merge(gr, on = 'product_id', how = 'left')

In [17]:
csv_file_path = 'test_.csv'  # Desired file name
test_.to_csv(csv_file_path, index=True) 